### Initialize MLflow tracking, emulating how Sientia website environment works

In [3]:
import sientia_tracker.simple_tracker as simple_tracker
import mlflow
import os


# Initialize MLflow tracking
tracking_uri = "file:./tmp/mlruns"
username = "example_user"
password = "example_password"
project_name = "example_project"

tracker = simple_tracker.SimpleTracker(tracking_uri,username,password)
tracker.set_project(project_name)



Experiment example_project already exists


### Load the dataset and create a model

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


### Initialize run. The only needed parameters are the experiment name and inputs

In [5]:
# Start a run
dataset_name= "Iris"
inputs= "sepal_length, sepal_width, petal_length, petal_width"
run = tracker.save_experiment(dataset_name=dataset_name, inputs=inputs)  #### TIVE QUE MUDAR AQUI ####
run_id = run.info.run_id


Saving experiment example_project


### Log models and metrics

In [6]:
# Log parameters and metrics
tracker.log_params({"max_iter": 1000})
tracker.log_metrics({"accuracy": accuracy})
# Log model
artifact_path = "LogisticRegression_for_Iris"
tracker.log_model(model, artifact_path)
# End the run
mlflow.end_run()

### Retrieve information of run

In [7]:
# Retrieve the run using the run ID
retrieved_run = tracker.client.get_run(run_id)

# Access and print metrics and params 
metrics = retrieved_run.data.metrics
params = retrieved_run.data.params
print("Metrics:", metrics)
for key, value in params.items():
    print( key,':' ,value)


model = tracker.client

Metrics: {'accuracy': 0.9666666666666667}
Dataset : Iris
Inputs : sepal_length, sepal_width, petal_length, petal_width
max_iter : 1000


# Retrieve the model


In [8]:
# Ensure the destiny path exists
dst_path="./local_model"
os.makedirs(dst_path, exist_ok=True)

# Download the model
local_path =  tracker.client.download_artifacts(run_id, artifact_path, dst_path="./local_model")
model = mlflow.pyfunc.load_model(local_path)

# Make predictions
print(model.predict(X_test))

[0 1 0 1 1 0 1 0 0 1 2 2 2 1 2 2 1 0 2 1 2 0 2 2 1 2 2 0 1 1]
